In [1]:
import pandas as pd
import gurobipy 
from gurobipy import Model, GRB, quicksum
import numpy as np

# Loading the dataset 
brick_index_value = pd.read_csv('./bricks_index_values.csv')
brick_rp_distances = pd.read_csv('./brick_rp_distances.csv')
#distances= pd.read_csv('./distances.xlsx')
pfizer =pd.read_csv('./Pfitzer10-100.csv', sep=';')

### 1. Objectif de minimisation de la distance

In [6]:
# Extracting data from the datasets
bricks = brick_index_value['brick'].tolist()
reps = list(range(1, 5))  # 4 représentants

# Converting distance to a dictionary f
distances = {
    row['brick']: [row[f'rp{i}'] for i in reps]
    for _, row in brick_rp_distances.iterrows()
}

# Converting index values to a dictionary
index_values = dict(zip(brick_index_value['brick'], brick_index_value['index_value']))

# Workload interval bounds
L = 0.8 
U = 1.2

# Modèle pour minimiser la distance
model_distance = Model("Minimize_Distance")

# Variables de décision 
# Variables binaires x[b, r], chaques briques b est assignée à un représentant r
x = model_distance.addVars(bricks, reps, vtype=GRB.BINARY, name="x") 

# Fonction objectif : minimiser la distance totale
model_distance.setObjective(
    quicksum(distances[b][r - 1] * x[b, r] for b in bricks for r in reps), GRB.MINIMIZE
)

# Fonction objectif : minimiser la distance totale
# Chaque brique est assignée à un seul représentant
model_distance.addConstrs(
    (quicksum(x[b, r] for r in reps) == 1 for b in bricks), name="AssignEachBrick"
)

# Contraintes de charge de travail dans l'intervalle [L, U]
model_distance.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) >= L for r in reps), name="MinWorkload"
)

model_distance.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) <= U for r in reps), name="MaxWorkload"
)

# Optimisation
model_distance.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 30 rows, 88 columns and 264 nonzeros
Model fingerprint: 0x4ea169f8
Variable types: 0 continuous, 88 integer (88 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [8e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+00]
Found heuristic solution: objective 353.0700000
Presolve time: 0.00s
Presolved: 30 rows, 88 columns, 264 nonzeros
Variable types: 0 continuous, 88 integer (88 binary)

Root relaxation: objective 1.488198e+02, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  148.81982    0    6

In [7]:
# Affichage des résultats
if model_distance.Status == GRB.OPTIMAL:
    print("\nDistance totale minimale :", model_distance.ObjVal)
    for b in bricks:
        for r in reps:
            if x[b, r].X > 0.5:
                print(f"Brick {b} -> Représentant {r}")



Distance totale minimale : 154.62
Brick 1 -> Représentant 4
Brick 2 -> Représentant 4
Brick 3 -> Représentant 4
Brick 4 -> Représentant 1
Brick 5 -> Représentant 1
Brick 6 -> Représentant 1
Brick 7 -> Représentant 1
Brick 8 -> Représentant 1
Brick 9 -> Représentant 1
Brick 10 -> Représentant 3
Brick 11 -> Représentant 2
Brick 12 -> Représentant 1
Brick 13 -> Représentant 2
Brick 14 -> Représentant 2
Brick 15 -> Représentant 3
Brick 16 -> Représentant 3
Brick 17 -> Représentant 3
Brick 18 -> Représentant 2
Brick 19 -> Représentant 1
Brick 20 -> Représentant 1
Brick 21 -> Représentant 4
Brick 22 -> Représentant 4


### 2. Objectif minimisation de disruption

In [9]:
# Assignation initiale des briques aux représentants
initial_assignment = {
    4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 15: 1,   # SR 1
    10: 2, 11: 2, 12: 2, 13: 2, 14: 2,     # SR 2
    9: 3, 16: 3, 17: 3, 18: 3,             # SR 3
    1: 4, 2: 4, 3: 4, 19: 4, 20: 4, 21: 4, 22: 4  # SR 4
}

# Modèle pour minimiser la perturbation
model_disruption = Model("Minimize_Disruption")

# Variables de décision
x = model_disruption.addVars(bricks, reps, vtype=GRB.BINARY, name="x")
y = model_disruption.addVars(bricks, reps, vtype=GRB.BINARY, name="y")

# Fonction objectif : minimiser la perturbation pondérée par l'index
model_disruption.setObjective(
    quicksum(index_values[b] * y[b, r] for b in bricks for r in reps), GRB.MINIMIZE
)

# Contraintes
# 1. Chaque brick doit être attribué à un seul représentant
model_disruption.addConstrs(
    (quicksum(x[b, r] for r in reps) == 1 for b in bricks), name="AssignEachBrick"
)

# Charge de travail dans l'intervalle [L, U]
model_disruption.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) >= L for r in reps), name="MinWorkload"
)
model_disruption.addConstrs(
    (quicksum(index_values[b] * x[b, r] for b in bricks) <= U for r in reps), name="MaxWorkload"
)

# 3. Mesurer la perturbation
model_disruption.addConstrs(
    (y[b, r] >= x[b, r] - (1 if initial_assignment[b] == r else 0)
     for b in bricks for r in reps), name="Disruption"
)

# Résoudre le modèle
model_disruption.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 118 rows, 176 columns and 440 nonzeros
Model fingerprint: 0xa736d90c
Variable types: 0 continuous, 176 integer (176 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [4e-02, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+00]
Found heuristic solution: objective 2.6088000
Presolve removed 88 rows and 88 columns
Presolve time: 0.00s
Presolved: 30 rows, 88 columns, 264 nonzeros
Variable types: 0 continuous, 88 integer (88 binary)

Root relaxation: objective 1.377000e-01, 25 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/No

In [10]:
# Afficher les résultats
if model_disruption.Status == GRB.OPTIMAL:
    print("\nPerturbation totale minimale :", model_disruption.ObjVal)
    for b in bricks:
        for r in reps:
            if x[b, r].X > 0.5:
                print(f"Brick {b} -> Représentant {r}")


Perturbation totale minimale : 0.1696
Brick 1 -> Représentant 4
Brick 2 -> Représentant 4
Brick 3 -> Représentant 4
Brick 4 -> Représentant 1
Brick 5 -> Représentant 1
Brick 6 -> Représentant 1
Brick 7 -> Représentant 1
Brick 8 -> Représentant 1
Brick 9 -> Représentant 3
Brick 10 -> Représentant 2
Brick 11 -> Représentant 3
Brick 12 -> Représentant 3
Brick 13 -> Représentant 2
Brick 14 -> Représentant 2
Brick 15 -> Représentant 1
Brick 16 -> Représentant 3
Brick 17 -> Représentant 3
Brick 18 -> Représentant 3
Brick 19 -> Représentant 4
Brick 20 -> Représentant 4
Brick 21 -> Représentant 4
Brick 22 -> Représentant 4
